In [169]:
%run tools
from tools import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [170]:
# carregando a base de dados
df = load_database()
# criando o modelo
vocabulary, model = bag_of_words(df['content'].values)
# vocabulary = tf_idf(df['content'].values)

In [171]:
X = vocabulary.toarray()
Y = df.label.values

classifier = AdaBoostClassifier(n_estimators=1000, learning_rate=0.5)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
#Treinar o classificador
classifier.fit(X_train, Y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.5,
                   n_estimators=1000, random_state=None)

In [172]:
#Rodar no conjunto de teste
predict_labels = classifier.predict(X_test)
#Avaliar o classificador
scores = evaluate(Y_test, predict_labels)

In [173]:
scores

(0.7160493827160493,
 0.6666666666666666,
 0.8205128205128205,
 0.7356321839080459)

In [167]:
STOPWORDS = set(stopwords.words("portuguese"))
def clean(text):
    d = re.sub("[^a-zA-Z]", " ", text)
    words = d.lower().split()
    words = [w for w in words if w not in STOPWORDS]
    words = ' '.join(words)
    return words

def content_page(url):
    req = requests.get(url)
    content = req.text
    s = BeautifulSoup(content, 'html.parser')
    data = s.find_all(text=True)
    data = ''.join(data)
    text = data.replace('\n', ' ').replace('\r', '').strip()
    text = ' '.join(text.split())
    return clean(text)

In [166]:
def classify_page(page):
    #Pegar o content de page
    text = content_page(page)
    #Rodar o classificador treinado para esta página
    features = model.transform([text])
    return classifier.predict(features.toarray())[0]